## Import Libraries and Connect to STK/ODTK

In [ ]:
# Import Python Libraries
# !pip install "C:\Program Files\AGI\STK 12\bin\AgPythonAPI\agi.stk12-12.2.0-py3-none-any.whl"
# !pip install opencv-python
import numpy as np
import pandas as pd
import cv2
import os
import shutil
import imageio
import matplotlib.image as mpimg 
import matplotlib.pyplot as plt 
import time
import win32com.client as w32c
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from skimage.color import rgb2gray
from scipy.spatial.transform import Rotation as R
from skimage.feature import peak_local_max
from astropy import units as u
from astropy.coordinates import Angle
%matplotlib inline

# Import STK Libraries

from agi.stk12.stkdesktop import STKDesktop
from agi.stk12.stkobjects import *
from agi.stk12.stkutil import *
from agi.stk12.vgt import *
from agi.stk12.utilities.colors import Color, Colors

# Import functions from EOIRProcessesing.py
from EOIRProcessingLib import *


In [ ]:
# Attatch to open instance of STK
stk = STKDesktop.AttachToApplication() # attach to exisiting instance of stk
root = stk.Root
root.Isolate()
root.UnitPreferences.Item('DateFormat').SetCurrentUnit('EpSec')
sc = root.CurrentScenario

# Attach to exisiting instance of ODTK
try:
    # 
    app  = w32c.GetActiveObject("ODTK7.Application")
    ODTK = app.Personality
except:
    print('Did not connect to ODTK')

## Examples and Inputs

Setup Considerations

* Use a sensor type of Fixed or Fixed in Axes if you are updating the sensor pointing based on observations
* Using updateSensorMethod = 'previousMeasurementDirection' requires azStart and elStart to be defined
* If your satellite is maneuevering or changing attitude, you might want to use a frame other than the body frame
* Depending on what type of sensor you are modeling, it may be beneficial to set constrains such as Sun exclusion angle, target must be lit, space only backgrounds, in STK and/or ODTK
* If you plan on running ODTK ensure STK and ODTK are using the same force models.See: https://help.agi.com/ODTK/index.htm#../LinkedDocuments/astrodynamicsConsistency.pdf 
* Right now the measurements passed to ODTK are assumed to be spacebased and passed in the form of a .geosc. The file format supports non-spacedbased measurements (facility observations), but the code to write the .geosc file would need to be updated to support this, see the function RADECToMeasurementFileLine in EOIRProcessingLib.py 
* You may want to modify the file paths of where images/videos are saved

In [ ]:
# Set Up
sensorPath = '*/Satellite/LEO/Sensor/LEOLaunchTracker'
tstart  = 0
tstop = 120
tstep = 2 

# Image Processing
minSNR = 3
percentofmax = 0.25
method = 'localpeaks' # 'localpeaks','minSNR','percentofmax','kmeans' 
maxObjects = 1
k = 1/3 # Common Values: 1/3(enhances dark spots), 1 (simply normalizes), and 2 (enhances bright spots)

# Update Sensor Position
updateSensorMethod = 'previousMeasurementDirection' # '' (this will skip updating), 'previousmeasurementdirection','odtk'
azStart = -90
elStart = 42

In [ ]:
# # Set Up
# sensorPath = '*/Satellite/LEO/Sensor/LEODetector'
# tstart  = 0
# tstop = 500
# tstep = 10

# # Image Processing
# minSNR = 3
# percentofmax = 0.25
# method = 'localpeaks' # 'localpeaks','minSNR','percentofmax','kmeans' 
# maxObjects = 1
# k = 1/3 # Common Values: 1/3(enhances dark spots), 1 (simply normalizes), and 2 (enhances bright spots)
# differenceImages = False # Only use for staring sensors


# # Update Sensor Position
# updateSensorMethod = '' # '' (this will skip updating), 'previousmeasurementdirection','odtk'

In [ ]:
# # Set Up
# sensorPath = '*/Aircraft/UAV/Sensor/HGVTracker'
# tstart  = 1000
# tstop = 1300
# tstep = 5 # 10

# # Image Processing
# minSNR = 2.5
# percentofmax = 0.25
# method = 'localpeaks' # 'localpeaks','minSNR','percentofmax','kmeans' 
# maxObjects = 1
# k = 1/3 # Common Values: 1/3(enhances dark spots), 1 (simply normalizes), and 2 (enhances bright spots)

# # Update Sensor Position
# updateSensorMethod = 'previousMeasurementDirection' # '' (this will skip updating), 'previousmeasurementdirection','odtk'
# azStart = 170
# elStart = 0

In [ ]:
# # Set Up
# sensorPath = '*/Satellite/HEO/Sensor/HEOLaunch'
# tstart  = 0
# tstop = 40
# tstep = 0.2

# # Image Processing
# minSNR = 5
# percentofmax = 0.25
# method = 'localpeaks' # 'localpeaks','minSNR','percentofmax','kmeans' 
# maxObjects = 1
# k = 1/3 # Common Values: 1/3(enhances dark spots), 1 (simply normalizes), and 2 (enhances bright spots)
# differenceImages = False # Only use for staring sensors
# deltaTime = 3 # Use a multiple of the time step

# # Update Sensor Position
# updateSensorMethod = '' # '' (this will skip updating), 'previousmeasurementdirection','odtk'


In [ ]:
# # Set Up, #Use the scal imagery
# sensorPath = '*/Aircraft/UAV/Sensor/LaunchTracker'
# tstart  = 0.1
# tstop = 60.1
# tstep = .5

# # Image Processing
# minSNR = 3
# percentofmax = 0.25
# method = 'localpeaks' # 'localpeaks','minSNR','percentofmax','kmeans' 
# maxObjects = 1
# k = 1/3 # Common Values: 1/3(enhances dark spots), 1 (simply normalizes), and 2 (enhances bright spots)

# # Update Sensor Position
# updateSensorMethod = 'previousMeasurementDirection' # '' (this will skip updating), 'previousmeasurementdirection','odtk'
# azStart = 100
# elStart = 6

In [ ]:
# # Set Up, #Use the scal imagery
# sensorPath = '*/Satellite/GEO/Sensor/Stare'
# tstart  = 0
# tstop = 500
# tstep = 10
# sensorPath = '*/Satellite/GEO/Sensor/StareHGV'
# tstart  = 1000
# tstop = 1400
# tstep = 10

# # Image Processing
# minSNR = 3
# percentofmax = 0.25
# method = 'localpeaks' # 'localpeaks','minSNR','percentofmax','kmeans' 
# maxObjects = 1
# k = 1/3 # Common Values: 1/3(enhances dark spots), 1 (simply normalizes), and 2 (enhances bright spots)

# # Update Sensor Position
# updateSensorMethod = '' # '' (this will skip updating), 'previousmeasurementdirection','odtk'


In [ ]:
# # Set Up
# sensorPath = '*/Satellite/RPO/Sensor/Detector'
# tstart  = 4800
# tstop = 12800
# tstep = 60

# # Image Processing
# minSNR = 5
# percentofmax = 0.25
# method = 'localpeaks' # 'localpeaks','minSNR','percentofmax','kmeans' 
# maxObjects = 1
# k = 1 # Common Values: 1/3(enhances dark spots), 1 (simply normalizes), and 2 (enhances bright spots)

# # Update Sensor Position
# updateSensorMethod = 'odtk'# '' (this will skip updating), 'previousmeasurementdirection','odtk'

In [ ]:
# Additional Inputs

# Output/Runtime Settings
writeVideo = False # Combine the tagged images into a video
realTimeRate = 5 # (tstop-tstart)/realTimeRate = videoLength assuming no frames are missing, how fast the final video should play relative to real time
reuseFiles = True # Reuse existing eoir images/text files if they already exist

# Limit Observations
useAccessConstraintsToLimitObs = False # Limit imaging times to be within access intervals
targetPath = '*/Satellite/TargetODTK' # Access object, usually the object you are trying to observe/detect

# Plotting
plotImage = True # Show the tagged images

# ODTK settings, Modify these as needed if you are running ODTK
ODSatName = 'RPO'  # Odtk satellite name with the sensor
ODFilterName = 'Filter1' # Filter name
ODTargetName = 'Target' # Object you are trying to perform OD on
targetID = 1001 # Target tracking ID in ODTK
sensorID = 1000 # Sensor tracking ID in ODTK
RAstd = 0.00 # 1 standard deviation in arcSec, format is xx.xx, setting to 0 uses ODTK's value
DECstd = 0.00 # 1 standard deviation in arcSec, format is xx.xx, setting to 0 uses ODTK's value
usePixelSizeForStd = False # overrides the RAstd and DECstd values based on pixel size
visualizeOD = False # Set to True if you want to see the filter run in STK as it goes


# Look for differences in images instead of the image itself, Only use for staring sensors
differenceImages = False
deltaTime = tstep # Time to difference images, use a multiple of the time step

## Execute Imaging Over Time

In [ ]:
# Get handles and setup for intiial config
sensor = root.GetObjectFromPath(sensorPath)
vectors = sensor.Vgt.Vectors
sensorName = sensorPath.split('/')[-1]

if differenceImages == True: # Differencing only works for staring images, don't update the sensor pointing
    updateSensorMethod = ''

# Use a sensor type of Fixed or Fixed in Axes if you are updating the sensor pointing
if sensor.PointingType == AgESnPointing.eSnPtFixedAxes:    
    axes = sensor.Pointing.ReferenceAxes[:-5]
else:
    axes = '/'.join(sensorPath.split('/')[1:3]) + ' Body'
    print('Assuming Parent Body axes and the sensor pointing type is Fixed')


# Set up ODTK
if updateSensorMethod.lower() == 'odtk':
    # Get ODTK handles
    ODScenario = ODTK.Scenario(0)
    ODSat = ODScenario.Satellite(ODSatName)
    ODFilter = ODScenario.Filter(ODFilterName)
    ODObject = ODScenario.Satellite(ODTargetName)
    # Add measurement file
    measurementFile = sensorName+".geosc"
    open(measurementFile, 'w').close() # Clear RADecfile
    measurementFiles = ODScenario.Measurements.Files
    measurementFiles.clear()
    newFile = measurementFiles.NewElem()
    newFile.Filename = os.getcwd()+'\\'+measurementFile
    measurementFiles.push_back(newFile)
    ODScenario.Measurements.Files(0).Enabled = True
    # Run filter with no Obs
    ODFilter.ProcessControl.StopMode = 'TimeSpan'
    ODFilter.ProcessControl.TimeSpan= (tstop-tstart)/3600
    ODFilter.Go()
    ODFilter.ProcessControl.StopMode = 'LastMeasurement'
    predictionTimeSpan = 3600 # sec 
    ODFilter.Output.STKEphemeris.Predict.TimeSpan = predictionTimeSpan/60  # Convert to mins  
    ephemerisFile = ODFilter.Output.STKEphemeris.Files(0).FileName()
    
    # Set up VGT
    points = sensor.Vgt.Points
    if not points.Contains('EstimatedTargetLocation'):
        point = points.Factory.Create('EstimatedTargetLocation','Estimated Pointing Location',AgECrdnPointType.eCrdnPointTypeFile)
        point.Filename = ephemerisFile
    point = points.Item('EstimatedTargetLocation')
    if not vectors.Contains('PointingDirection'):
        vector = vectors.Factory.Create('PointingDirection','Estimated Pointing Direction',AgECrdnVectorType.eCrdnVectorTypeDisplacement)
        vector.Destination.SetPath(sensorPath[2:]+' EstimatedTargetLocation')
    
    # Set up filter run visualization
    if visualizeOD == True:
        if sc.Children.Contains(AgESTKObjectType.eSatellite,targetPath.split('/')[-1]+'Filter'):
            satVis = root.GetObjectFromPath(targetPath+'Filter')
        else:
            satVis = sc.Children.New(AgESTKObjectType.eSatellite,targetPath.split('/')[-1]+'Filter')
        satVis.SetPropagatorType(AgEVePropagatorType.ePropagatorStkExternal)
        satVis.Propagator.Filename = ephemerisFile
        satVis.VO.Covariance.Attributes.IsVisible = True
        satVis.VO.OrbitSystems.RemoveAll()
        satVis.VO.OrbitSystems.InertialByWindow.IsVisible = False
        satVis.VO.OrbitSystems.Add(targetPath.split('*/')[-1]+' RIC System')



# Set up measurement vector
if not vectors.Contains('MeasurementDirection'):
    vectors.Factory.Create('MeasurementDirection','Measurement Direction',AgECrdnVectorType.eCrdnVectorTypeFixedInAxes)
vector = vectors.Item('MeasurementDirection')
vector.ReferenceAxes.SetPath(axes)

# Update initial pointing
if updateSensorMethod.lower() == 'previousmeasurementdirection':
    sensor.Pointing.Orientation.AssignAzEl(azStart,elStart,AgEAzElAboutBoresight.eAzElAboutBoresightRotate)
    
# Get the access handle
if useAccessConstraintsToLimitObs == True:
    access = sensor.GetAccess(targetPath)

# Get pixel size
horizontalPixels,verticalPixels,horizontalAngle,verticalAngle  = getSensorFOVAndPixels(sensor)
horizontalDegPerPixel = horizontalAngle/horizontalPixels
verticalDegPerPixel = verticalAngle/verticalPixels
horizontalCenter = horizontalPixels/2+0.5
verticalCenter = verticalPixels/2+0.5
# Update RA and Dec based on pixel size
if usePixelSizeForStd == True:
    RAstd = horizontalDegPerPixel*3600
    DECstd = verticalDegPerPixel*3600  # Be careful if this exceeds 99 arc Seconds, should add a check, could also leave blank to use ODTK values
    if RAstd >=100 or DECstd >=100:
        print('Setting standard deviations to use ODTK values')
        RAstd = 00.0
        DECstd = 00.0

# Set up times and measurements
times = np.arange(tstart,tstop,tstep)
times = np.append(times,tstop).round(3) # round to 1/1000th of a sec get rid of numerical issues
timesWithTaggedImages = []
measurementsAzElErrors = []
meaurementsAzEl = []
sensorPointingHistory = []
imageFolder = os.getcwd()+'\\Images'
t1 = time.time()

# Loop through time
for t in times:
    # Set Animation time
    print('Time:',t)
    t = float(t)
    root.CurrentTime = t
    
    # If running ODTK, update sensor pointing to predicted target location
    if updateSensorMethod.lower() == 'odtk':
        # Have to force a filereload, there may be a better way, or could switch between the copy to save a bit of time
        shutil.copyfile(ephemerisFile, ephemerisFile.split('.')[0]+'Copy.e') #copy src to dst
        point.Filename = ephemerisFile.split('.')[0]+'Copy.e'
        point.Filename = ephemerisFile
        if visualizeOD == True:
            satVis.Propagator.Filename = ephemerisFile.split('.')[0]+'Copy.e'
            satVis.Propagator.Filename = ephemerisFile
            satVis.Propagator.Propagate()
        _,az,el = getPointingDirection(sensor,t,t,tstep,axes=axes)
        sensor.Pointing.Orientation.AssignAzEl(az,el,AgEAzElAboutBoresight.eAzElAboutBoresightRotate)
        print('Pointing Update:',az,el)
        sensorPointingHistory.append((t,az,el))
   
    # Limit imagine times to when the object has access, this will take into account pointing updates
    if useAccessConstraintsToLimitObs == True:
        access.SpecifyAccessTimePeriod(t,t)
        access.ComputeAccess()
        skipAccess = False if access.ComputedAccessIntervalTimes.Count != 0 else True
    else:
        skipAccess = False
        
    # Image Processing and Tagging
    if skipAccess == False:
        # Construct file names
        imageName = '"{}\\{}Time{}.jpg"'.format(imageFolder,sensorName,str(t).replace('.','_'))
        textName = '"{}\\{}Time{}.txt"'.format(imageFolder,sensorName,str(t).replace('.','_'))

        # Run EOIR
        getEOIRImages(root,sensorPath,imageName='',textName=textName,reuseFiles=reuseFiles) # set imageName=imageName if you want EOIR to generate the image

        # Image processing
        data = np.loadtxt(textName.replace('"','')) # Load EOIR Image
        # Difference Images if needed
        if differenceImages == True:
            textNamePrevious = '"{}\\{}Time{}.txt"'.format(imageFolder,sensorName,str(t-deltaTime).replace('.','_'))
            if os.path.exists(textNamePrevious.replace('"','')):
                dataPrevious = np.loadtxt(textNamePrevious.replace('"',''))
                data = data-dataPrevious
            else:
                continue # skip to the next iteration if not previous image exists
        timesWithTaggedImages.append(t) # Time where an image processing attempt was made 
        image = normalizeImage(data,k=k,convertToInt=False,plotImage=False)

        # Get the object centers
        maxSNR = getMaxSNR(image)
        objectCenters = getObjectCenters(image,method=method,minSNR=minSNR,percentofmax=percentofmax,maxObjects=maxObjects)
        if len(objectCenters) > 0:
            objectCenters,objectSNRS = sortBySNR(image,objectCenters) # Puts highest SNR last
        else: 
            objectSNRS = []
        print('Object Centers:',objectCenters)
        print('Object SNRs:',objectSNRS)

        # Store Az El measurements
        for objectCenter in objectCenters:
            azError = (objectCenter[1]-horizontalCenter)*horizontalDegPerPixel
            elError = (objectCenter[0]-verticalCenter)*verticalDegPerPixel
            measurementsAzElErrors.append((t,azError,elError))
            
        # Update pointing and measurements if an object is detected
        if len(objectCenters) > 0:
            print('Errors (deg): ',azError,elError)

            # Obs Association: The image detection process can detect multiple objects and figuring out which obs corresponds to which object is non-trival. This is an area of future improvment, with Python or lettings ODTK/OAT help out
            # Common issues: Mistags, missed observations/undetectable signals,false positives (find peaks which are not the actual object, stars, clouds, background clutter.) Be aware of this, better image processing or sensors could resolve this.
            # For now the script assumes a very simple object association: the highest SNR is the specified target object and will use this for pointing updates 

            # Get current rotation matrix for sensor body frame to specified axes
            timeRotationZYXs = computeSensorBodyToParentRotations(sensor,t,t,tstep,axes=axes)
            # Can also compute true values aif desired
    #         trueAzElError = computeTrueSensorAzElError(sensor,t,t,tstep,target=targetPath.split('/')[-1])

            # Calculate target direction in axes from az el offsets in sensor image
            azMeas,elMeas,targetVec = updatePointingDir(measurementsAzElErrors[-1][1],measurementsAzElErrors[-1][2],timeRotationZYXs[0,:])
            meaurementsAzEl.append((t,azMeas,elMeas))

            # Update the measurement direction vector
            vector.Direction.AssignXYZ(targetVec[0],targetVec[1],targetVec[2])

            # Convert the measurement direction vector to RA and Dec measurements
            if updateSensorMethod.lower() == 'odtk':
                _,ra,dec = getRADECMeasurements(sensor,t,t,tstep,useMeasurementDirection=True)
                with open(measurementFile,"a+") as f:
                    line = RADECToMeasurementFileLine(root,t,ra,dec,targetID=1001,sensorID=1000,RAstd=RAstd,DECstd=DECstd)
                    f.write(line)
                print('RA Dec:',ra,dec)
                ODFilter.Go()

            if updateSensorMethod.lower() == 'previousmeasurementdirection':
                sensor.Pointing.Orientation.AssignAzEl(azMeas,elMeas,AgEAzElAboutBoresight.eAzElAboutBoresightRotate)
                sensorPointingHistory.append((t,azMeas,elMeas))
                # Alternative approaches/future improvements: could estimate position and velocity, can do simple regression for velocity, could look at multiple images(over time and different sensors), could do missle model and kalman filter
        
        # Plotting
        if plotImage == True:
            plt.figure(figsize=(8,8))
            if differenceImages == True:
                data = np.loadtxt(textName.replace('"','')) # reload data for saving the image
                image = normalizeImage(data,k=k,convertToInt=False,plotImage=False)
            plt.imshow(image)
            plt.imshow(image,cmap='gray')
            if len(objectCenters)>0:
                plt.plot(objectCenters[:,1],objectCenters[:,0],'ro',markersize=12,markerfacecolor='none')
            plt.title('Time: '+str(t))
            plt.savefig(imageName.replace('"','').split('.')[0]+'Tagged.png',dpi=500)
            plt.show()

        # continue loop
        print('RunTime: ',time.time()-t1)
        
    else:
        print('Skipped Time:',t) # No access


# Create a history of sensor pointing
writeSensorPointingFile(sensorPointingHistory,fileName='{}SensorPointing.sp'.format(sensorName),axes=axes)
print('Wrote ','{}SensorPointing.sp'.format(sensorName))

# Create a video
if writeVideo == True: 
    createVideo(sensorName,timesWithTaggedImages,tstep,imageFolder,realTimeRate=realTimeRate)
    print('Wrote ','{}.mp4'.format(sensorName))
print(time.time()-t1)    
    